# 6_day45_pyspark_read_json
####### JSON types: single-line (one object per line), multi-line (one object spans lines), complex (nested arrays/objects; fields vary per record).---> single-line json, multi-line json and complex json

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate

### 1. This reads a multi-line JSON file


In [0]:
df = spark.read.json("/Volumes/workspace/default/august_2025/json_files/sample1.json")
df.display()      # here defult is multiLine = False
df.printSchema()

In [0]:
df = spark.read.json("/Volumes/workspace/default/august_2025/json_files/sample1.json", multiLine=True)
df.display()

df.printSchema()

### 2. This reads a Single-line JSON file

In [0]:
df2 = spark.read.json("/Volumes/workspace/default/august_2025/json_files/singleline.json", multiLine=False)
df2.display()        # here not write to need 'multiLine = False', because of defulat value. 
df2.printSchema() 

### 3. This reads a complex(nested) JSON file

In [0]:
df3 = spark.read.json("/Volumes/workspace/default/august_2025/json_files/Complex.json", multiLine=True)
df3.display()
df3.printSchema()

In [0]:
# After data read from json files check for Array type and Struct type
# if we have Array type column, Do explode/explode outer
# then check again schema if still array type columns present do explode again
# check if any struct type then extract each field from struct field

In [0]:
from pyspark.sql.functions import explode
df3_explore = df3.select("*", explode("phoneNumbers").alias("phone"))
df3_explore.display() 

##### phoneNumbers: array (nullable = true)
#######  if we have Array type column, Do explode/explode outer

###### After using explode() on the phone column, each phone entry (home, office, etc.) becomes a separate row. Once expanded, you can remove the original nested column using df = df.drop("phone").

In [0]:
from pyspark.sql.functions import explode
df3_explore = df3.select("*", explode("phoneNumbers").alias("phone")).drop("phoneNumbers")
df3_explore.display()
df3_explore.printSchema()


##### Struct
###### address: struct (nullable = true)
###### phone: struct (nullable = true)
###### now no array type column
###### # check if any struct type then extract each field from struct field

In [0]:
df3_extract = df3_explore.select('address.city', 
                                 'address.state',
                                  'address.streetAddress',
                                   'age', 'firstName',
                                    'gender',
                                     'lastName',
                                      'phone.number',
                                      'phone.type' )
df3_extract.display()
df3_extract.printSchema()                         


####### After exploding the phone array, use a pivot transformation to bring all phone types (home, office, etc.) back into one row — each as a separate column like home_1, office_1, etc. This makes it a one-to-one structured dataset again.
####### If QA doesn’t want multiple rows for one person, pivot the data — make each phone/address type a separate column (like home_1, office_1, etc.) so that all info stays in one row per person.

### “What is the difference between explode() and explode_outer() in PySpark?”
#### explode() → ignores null values (skips them).
#### explode_outer() → includes null values as separate rows.
#### 👉 Use explode() to skip nulls, explode_outer() to keep them.

In [0]:
from pyspark.sql.functions import explode,first

df_final = df3_extract.groupBy('city', 'state', 'streetAddress', 'age', 'firstName', 'gender', 'lastName','age','lastName').pivot('type').agg(first('number'))
df_final.display()
df_final.printSchema()

### Same complex jason

In [0]:
df4 = spark.read.format('json').option('multiLine', True).load("/Volumes/workspace/default/august_2025/json_files/Complex2.json")

df4.display()

df4.printSchema()

In [0]:
df4_explode = df4.select("*", explode('people').alias('person')).drop('people')

df4_explode.display()

df4_explode.printSchema()

In [0]:
df4_final = df4_explode.select('person.age',
                               "person.email",
                               "person.firstName",
                               "person.gender",
                               "person.lastName",
                               "person.number")

df4_final.display()
df4_final.printSchema()

In [0]:
df_final.filter('age>25').display()